In [1]:
#required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [18]:
hud_data = pd.read_csv(r'C:\Users\willd\OneDrive\Desktop\DataAnalysis\python data\Census\HUD_COC_DATA.csv')

In [10]:
hud_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Columns: 332 entries, year to rural
dtypes: float64(257), int64(73), object(2)
memory usage: 7.6+ MB


In [12]:
hud_data.shape

(3008, 332)

In [13]:
list(hud_data.columns)

['year',
 'cocnumber',
 'pit_tot_shelt_pit_hud',
 'pit_tot_unshelt_pit_hud',
 'pit_tot_hless_pit_hud',
 'pit_ind_shelt_pit_hud',
 'pit_ind_unshelt_pit_hud',
 'pit_ind_hless_pit_hud',
 'pit_perfam_shelt_pit_hud',
 'pit_perfam_unshelt_pit_hud',
 'pit_perfam_hless_pit_hud',
 'pit_ind_chronic_hless_pit_hud',
 'pit_perfam_chronic_hless_pit_hud',
 'pit_vet_hless_pit_hud',
 'econ_urb_urbanicity',
 'coctag',
 'panelvar',
 'hou_pol_totalind_hud',
 'hou_pol_totalday_hud',
 'hou_pol_totalexit_hud',
 'hou_pol_numret6mos_hud',
 'hou_pol_numret12mos_hud',
 'hou_pol_fedfundcoc',
 'hou_pol_fund_project',
 'hou_pol_bed_es_hic_hud',
 'hou_pol_bed_oph_hic_hud',
 'hou_pol_bed_psh_hic_hud',
 'hou_pol_bed_rrh_hic_hud',
 'hou_pol_bed_sh_hic_hud',
 'hou_pol_bed_th_hic_hud',
 'hou_pol_perm_bed_hic_hud',
 'hou_pol_temp_bed_hic_hud',
 'dem_pop_pop_census',
 'dem_pop_male_census',
 'dem_pop_female_census',
 'dem_pop_child_census',
 'dem_pop_adult_census',
 'dem_pop_senior_census',
 'dem_soc_white_census',
 'dem_s

In [19]:
hud_data['cocnumber'] = hud_data['cocnumber'].str[:2]

In [20]:
hud_data.head(5)

,year,cocnumber,pit_tot_shelt_pit_hud,pit_tot_unshelt_pit_hud,pit_tot_hless_pit_hud,pit_ind_shelt_pit_hud,pit_ind_unshelt_pit_hud,pit_ind_hless_pit_hud,pit_perfam_shelt_pit_hud,pit_perfam_unshelt_pit_hud,...,sub_high_cost_rent75,sub_high_cost_homeval75,sub_high_rent_share75,tight_high_cost_rental_mkt,sub_tight_high_cost_rent,sub_west_coast_all_urb,sub_west_census,major_city,suburban,rural
0,2010,AK,1113.0,118.0,1231.0,633.0,107.0,740.0,480.0,11.0,...,1,1,1,3,1,1,1,1,0,0
1,2011,AK,1082.0,141.0,1223.0,677.0,117.0,794.0,405.0,24.0,...,1,1,0,3,1,1,1,1,0,0
2,2012,AK,1097.0,50.0,1147.0,756.0,35.0,791.0,341.0,15.0,...,1,1,1,3,1,1,1,1,0,0
3,2013,AK,1070.0,52.0,1122.0,792.0,52.0,844.0,278.0,0.0,...,1,1,0,3,1,1,1,1,0,0
4,2014,AK,970.0,53.0,1023.0,688.0,48.0,736.0,282.0,5.0,...,1,1,1,3,1,1,1,1,0,0


In [ ]:
state_totals = 

In [ ]:
ach_cols = ['Building IRN', 'Performance Index Percent']
df_df = df_df[df_cols]